<a href="https://colab.research.google.com/github/Mahdi-Golizadeh/Natural-Language-Processing/blob/main/transformers/token_classification/Token_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install & Import necessary libraries

In [113]:
!pip install datasets
!pip install transformers
!pip install poseval
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement poseval (from versions: none)
ERROR: No matching distribution found for poseval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [114]:
import datasets
import transformers
import evaluate
import numpy as np

## Dataset

In [115]:
raw_datasets = datasets.load_dataset("conll2003")

  0%|          | 0/3 [00:00<?, ?it/s]

In [116]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [117]:
raw_datasets["train"][0]["tokens"], raw_datasets["train"][0]["pos_tags"]

(['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 [22, 42, 16, 21, 35, 37, 16, 21, 7])

In [118]:
pos_features = raw_datasets["train"].features["pos_tags"]

In [119]:
pos_features

Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'], id=None), length=-1, id=None)

In [120]:
label_names = pos_features.feature.names

In [121]:
names = pos_features.feature.names
words = raw_datasets["train"][5]["tokens"]
labels = raw_datasets["train"][5]["pos_tags"]
l_1 = ""
l_2 = ""
for word, label in zip(words, labels):
    label_name = names[label]
    max_length = max(len(word), len(label_name))
    l_1 += word + " " * (max_length - len(word) + 1)
    if label_name in ['"', "''", '#',  '$',  '(',  ')', ',', '.', ':', '``',]:
        l_2 +=" " * (max_length + 1)
    else:
        l_2 += label_name + " " * (max_length - len(label_name) + 1)

print(l_1, l_2, sep= "\n")

" We  do  n't support any such recommendation because we  do  n't see any grounds for it  , " the Commission 's  chief spokesman Nikolaus van der Pas told a  news briefing . 
  PRP VBP RB  VB      DT  JJ   NN             IN      PRP VBP RB  VB  DT  NNS     IN  PRP     DT  NNP        POS JJ    NN        NNP      NNP FW  NNP VBD  DT NN   NN         


In [122]:
checkpoint = "bert-base-cased"

In [123]:
tokenizer = transformers.AutoTokenizer.from_pretrained(checkpoint)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/vocab.txt
loading file tokenize

In [124]:
tokenizer.is_fast


True

In [125]:
inputs = tokenizer(raw_datasets["train"][2]["tokens"],
                   is_split_into_words= True)

In [126]:
inputs

{'input_ids': [101, 26660, 13329, 12649, 15928, 1820, 118, 4775, 118, 1659, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [127]:
inputs.tokens()

['[CLS]', 'BR', '##US', '##SE', '##LS', '1996', '-', '08', '-', '22', '[SEP]']

In [128]:
inputs.word_ids()

[None, 0, 0, 0, 0, 1, 1, 1, 1, 1, None]

we are required to convert special tokens into -100 in order to be ignored by loss function

In [129]:
def special_token_care(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
    return new_labels

In [130]:
def tokenize_and_care(example):
    tokenized_input = tokenizer(example["tokens"],
                                truncation= True,
                                is_split_into_words= True)
    all_labels = example["pos_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_input.word_ids(i)
        new_labels.append(special_token_care(labels, word_ids))
        tokenized_input["labels"] = new_labels
    return tokenized_input

In [131]:
tokenized_datasets = raw_datasets.map(tokenize_and_care, 
                                      batched= True,
                                      remove_columns= raw_datasets["train"].column_names)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [132]:
tokenized_datasets["train"][12]

{'input_ids': [101,
  2809,
  1699,
  1105,
  2855,
  5534,
  17355,
  9022,
  2879,
  112,
  188,
  5835,
  119,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 30, 22, 10, 22, 38, 22, 27, 21, 7, -100]}

In [133]:
data_collator = transformers.DataCollatorForTokenClassification(tokenizer= tokenizer)

In [134]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [135]:
batch["input_ids"]

tensor([[  101,  7270, 22961,  1528,  1840,  1106, 21423,  1418,  2495, 12913,
           119,   102],
        [  101,  1943, 14428,   102,     0,     0,     0,     0,     0,     0,
             0,     0]])

In [136]:
metric = evaluate.load("poseval")

In [137]:
metric

EvaluationModule(name: "poseval", module_type: "metric", features: {'predictions': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='label'), length=-1, id='sequence')}, usage: """
Computes the poseval metric.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    zero_division: Which value to substitute as a metric value when encountering zero division. Should be on of 0, 1,
        "warn". "warn" acts as 0, but the warning is raised.

Returns:
    'scores': dict. Summary of the scores for overall and per type
        Overall (weighted and macro avg):
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': F1 score, also known as balanced F-score or F-measure,
        Per type:
            'precision': p

In [162]:
def compute_metric(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis= -1)
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p,l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    metrics = metric.compute(predictions= true_predictions,
                             references= true_labels)
    return {"precision": metrics["weighted avg"]["precision"],
            "recall": metrics["weighted avg"]["recall"],
            "f1":metrics["weighted avg"]["f1-score"],}

In [163]:
labels = raw_datasets["train"][0]["pos_tags"]
labels = [label_names[i] for i in labels]
predictions = labels.copy()
predictions[2] = "VB"
result = metric.compute(predictions= [predictions],
               references= [labels])

In [164]:
result["weighted avg"]

{'precision': 0.9444444444444444,
 'recall': 0.8888888888888888,
 'f1-score': 0.8888888888888888,
 'support': 9}

In [165]:
id2labels = {i: label for i, label in enumerate(label_names)}
label2ids = {v: k for k, v in id2labels.items()}

In [166]:
model = transformers.AutoModelForTokenClassification.from_pretrained(checkpoint,
                                                                     id2label=id2labels,
                                                                     label2id= label2ids)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/5532cc56f74641d4bb33641f5c76a55d11f846e0/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\"",
    "1": "''",
    "2": "#",
    "3": "$",
    "4": "(",
    "5": ")",
    "6": ",",
    "7": ".",
    "8": ":",
    "9": "``",
    "10": "CC",
    "11": "CD",
    "12": "DT",
    "13": "EX",
    "14": "FW",
    "15": "IN",
    "16": "JJ",
    "17": "JJR",
    "18": "JJS",
    "19": "LS",
    "20": "MD",
    "21": "NN",
    "22": "NNP",
    "23": "NNPS",
    "24": "NNS",
    "25": "NN|SYM",
    "26": "PDT",
    "27": "POS",
    "28": "PRP",
    "29": "PRP$",
    "30": "RB",
    "31": "RBR",
    "32": "R

In [167]:
model.config.num_labels

47

In [168]:
len(id2labels)

47

In [169]:
args = transformers.TrainingArguments("bert-finetuned-pos",
                                      evaluation_strategy= "epoch",
                                      save_strategy= "epoch",
                                      save_total_limit= 2,
                                      fp16= True,
                                      learning_rate= 2e-5,
                                      num_train_epochs= 2,
                                      weight_decay= .01)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [170]:
trainer = transformers.Trainer(model= model,
                               args= args,
                               train_dataset= tokenized_datasets["train"],
                               eval_dataset= tokenized_datasets["validation"],
                               data_collator= data_collator,
                               compute_metrics= compute_metric,
                               tokenizer= tokenizer)

Using cuda_amp half precision backend


In [171]:
example = tokenized_datasets["test"][0]

In [172]:
# model(**example)

In [173]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14041
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3512
  Number of trainable parameters = 107755823


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,1.148500,0.848467,0.686173,0.684455,0.679757
2,0.737900,0.653457,0.757103,0.759122,0.754692


***** Running Evaluation *****
  Num examples = 3250
  Batch size = 8
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

TrainOutput(global_step=3512, training_loss=1.1559048687406988, metrics={'train_runtime': 386.421, 'train_samples_per_second': 72.672, 'train_steps_per_second': 9.089, 'total_flos': 615901866378294.0, 'train_loss': 1.1559048687406988, 'epoch': 2.0})

In [48]:
pos_tagger = transformers.pipeline("token-classification", model= "/content/bert-finetuned-pos/checkpoint-3512")

loading configuration file /content/bert-finetuned-pos/checkpoint-3512/config.json
Model config BertConfig {
  "_name_or_path": "/content/bert-finetuned-pos/checkpoint-3512",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\"",
    "1": "''",
    "2": "#",
    "3": "$",
    "4": "(",
    "5": ")",
    "6": ",",
    "7": ".",
    "8": ":",
    "9": "``",
    "10": "CC",
    "11": "CD",
    "12": "DT",
    "13": "EX",
    "14": "FW",
    "15": "IN",
    "16": "JJ",
    "17": "JJR",
    "18": "JJS",
    "19": "LS",
    "20": "MD",
    "21": "NN",
    "22": "NNP",
    "23": "NNPS",
    "24": "NNS",
    "25": "NN|SYM",
    "26": "PDT",
    "27": "POS",
    "28": "PRP",
    "29": "PRP$",
    "30": "RB",
    "31": "RBR",
    "32": "RBS",
    "33": "RP",
    "34": "SYM",
    "35":

In [49]:
pos_tagger("I'm going to sleep.")

[{'entity': 'PRP',
  'score': 0.98948157,
  'index': 1,
  'word': 'I',
  'start': 0,
  'end': 1},
 {'entity': 'VBP',
  'score': 0.8766078,
  'index': 2,
  'word': "'",
  'start': 1,
  'end': 2},
 {'entity': 'VBG',
  'score': 0.941468,
  'index': 3,
  'word': 'm',
  'start': 2,
  'end': 3},
 {'entity': 'TO',
  'score': 0.96362334,
  'index': 4,
  'word': 'going',
  'start': 4,
  'end': 9},
 {'entity': 'VB',
  'score': 0.8158369,
  'index': 5,
  'word': 'to',
  'start': 10,
  'end': 12},
 {'entity': '.',
  'score': 0.9957366,
  'index': 6,
  'word': 'sleep',
  'start': 13,
  'end': 18},
 {'entity': '.',
  'score': 0.99788505,
  'index': 7,
  'word': '.',
  'start': 18,
  'end': 19}]

In [56]:
test = tokenized_datasets["test"].set_format("torch")

In [58]:
model(**tokenized_datasets["train"][0])

AttributeError: ignored

In [70]:
import torch
test_dataloader = torch.utils.data.DataLoader(tokenized_datasets["test"],
                                            #    shuffle= True,
                                               collate_fn= data_collator,
                                               batch_size= 8
                                              )

In [71]:
for batch in test_dataloader:
    out = model(**batch.to("cuda"))

In [78]:
labels = tokenized_datasets["test"][-5:]["labels"]
logit = out.logits

In [79]:
labels

[tensor([-100,   12,   42,   46,   12,   42,   30,   16,   12,   21,   15,   28,
            6,    0,   22,   38,    7, -100]),
 tensor([-100,    0,   28,   38,   12,   21,   15,   28,   12,   38,   15,   12,
           21,    6,   15,   28,   38,   15,   24,   15,   28,   38,   35,   37,
            6,   15,   28,   21,   18,    6,    0,   28,   38,    7, -100]),
 tensor([-100,   22,   38,   22,   15,   11,   24,    6,   15,   43,   21,   28,
           38,   30,   11,   24,   15,   30,   11,   24,   15,   28,   38,   15,
           22,   11,    7, -100]),
 tensor([-100,   28,   38,   22,   35,   11,   16,   22,   22,   24,   24,   10,
           35,   12,   11,   16,   21,   24,   15,   22,    6,   15,   12,   16,
           21,   12,   16,   22,   21,   11,   15,   29,   21,   21,    7, -100]),
 tensor([-100,   12,   16,   16,   22,   22,   21,   38,   30,   37,   29,   22,
           21,   15,   12,   21,   15,   11,   10,   30,   40,   11,   24,   10,
           38,   12,   16,   

In [84]:
def compute_metric(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis= -1)
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [[label_names[p] for (p,l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    metrics = metric.compute(predictions= true_predictions,
                             references= true_labels)
    return {"precision": metrics["weighted avg"]["precision"],
            "recall": metrics["weighted avg"]["recall"],
            "f1":metrics["weighted avg"]["f1_score"],}

TypeError: ignored

In [85]:
labels = tokenized_datasets["test"][-5:]["labels"]
logit = out.logits.cpu().detach().numpy()

In [86]:
predictions = np.argmax(logit, axis= -1)

In [88]:
predictions.shape

(5, 44)

In [95]:
labels[4].shape

torch.Size([43])

In [96]:
true_labels = [[label_names[l] for l in label if l != -100] for label in labels]

In [97]:
true_labels

[['DT',
  'VBZ',
  'WRB',
  'DT',
  'VBZ',
  'RB',
  'JJ',
  'DT',
  'NN',
  'IN',
  'PRP',
  ',',
  '"',
  'NNP',
  'VBD',
  '.'],
 ['"',
  'PRP',
  'VBD',
  'DT',
  'NN',
  'IN',
  'PRP',
  'DT',
  'VBD',
  'IN',
  'DT',
  'NN',
  ',',
  'IN',
  'PRP',
  'VBD',
  'IN',
  'NNS',
  'IN',
  'PRP',
  'VBD',
  'TO',
  'VB',
  ',',
  'IN',
  'PRP',
  'NN',
  'JJS',
  ',',
  '"',
  'PRP',
  'VBD',
  '.'],
 ['NNP',
  'VBD',
  'NNP',
  'IN',
  'CD',
  'NNS',
  ',',
  'IN',
  'WDT',
  'NN',
  'PRP',
  'VBD',
  'RB',
  'CD',
  'NNS',
  'IN',
  'RB',
  'CD',
  'NNS',
  'IN',
  'PRP',
  'VBD',
  'IN',
  'NNP',
  'CD',
  '.'],
 ['PRP',
  'VBD',
  'NNP',
  'TO',
  'CD',
  'JJ',
  'NNP',
  'NNP',
  'NNS',
  'NNS',
  'CC',
  'TO',
  'DT',
  'CD',
  'JJ',
  'NN',
  'NNS',
  'IN',
  'NNP',
  ',',
  'IN',
  'DT',
  'JJ',
  'NN',
  'DT',
  'JJ',
  'NNP',
  'NN',
  'CD',
  'IN',
  'PRP$',
  'NN',
  'NN',
  '.'],
 ['DT',
  'JJ',
  'JJ',
  'NNP',
  'NNP',
  'NN',
  'VBD',
  'RB',
  'VB',
  'PRP$',
  'NNP',


In [98]:
true_predictions = [[label_names[p] for (p,l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

In [99]:
true_predictions

[['DT',
  'VBZ',
  'WRB',
  'DT',
  'VBZ',
  'RB',
  'JJ',
  'DT',
  'NN',
  'IN',
  'PRP',
  ',',
  '"',
  'NNP',
  'VBD',
  '.'],
 ['"',
  'PRP',
  'VBD',
  'DT',
  'NN',
  'IN',
  'PRP',
  'RB',
  'VBD',
  'IN',
  'DT',
  'NN',
  ',',
  'IN',
  'PRP',
  'VBD',
  'IN',
  'NNS',
  'IN',
  'PRP',
  'VBD',
  'TO',
  'VB',
  ',',
  'IN',
  'PRP',
  'VBP',
  'JJS',
  ',',
  '"',
  'PRP',
  'VBD',
  '.'],
 ['NNP',
  'VBD',
  'NNP',
  'IN',
  'CD',
  'NNS',
  ',',
  'IN',
  'WDT',
  'NN',
  'PRP',
  'VBD',
  'RB',
  'CD',
  'NNS',
  'IN',
  'RB',
  'CD',
  'NNS',
  'IN',
  'PRP',
  'VBD',
  'IN',
  'NNP',
  'CD',
  '.'],
 ['PRP',
  'VBD',
  'NNP',
  'TO',
  'CD',
  'JJ',
  'NNP',
  'NNP',
  'NNS',
  'NNS',
  'CC',
  'TO',
  'DT',
  'CD',
  'JJ',
  'NN',
  'NNS',
  'IN',
  'NNP',
  ',',
  'IN',
  'DT',
  'NNP',
  'VBD',
  'DT',
  'JJ',
  'NNP',
  'NNP',
  'NNP',
  'IN',
  'PRP$',
  'NN',
  'NN',
  '.'],
 ['DT',
  'JJ',
  'JJ',
  'NNP',
  'NNP',
  'NN',
  'VBD',
  'RB',
  'VB',
  'PRP$',
  'N

In [112]:
metrics = metric.compute(predictions= true_predictions,references= true_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

In [106]:
true_predictions[0], true_labels[0]

(['DT',
  'VBZ',
  'WRB',
  'DT',
  'VBZ',
  'RB',
  'JJ',
  'DT',
  'NN',
  'IN',
  'PRP',
  ',',
  '"',
  'NNP',
  'VBD',
  '.'],
 ['DT',
  'VBZ',
  'WRB',
  'DT',
  'VBZ',
  'RB',
  'JJ',
  'DT',
  'NN',
  'IN',
  'PRP',
  ',',
  '"',
  'NNP',
  'VBD',
  '.'])

In [111]:
metric.compute(predictions=true_predictions, references=true_labels)

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1

{'"': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3},
 ',': {'precision': 0.875,
  'recall': 1.0,
  'f1-score': 0.9333333333333333,
  'support': 7},
 '.': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5},
 'CC': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 3},
 'CD': {'precision': 1.0,
  'recall': 0.8,
  'f1-score': 0.888888888888889,
  'support': 10},
 'DT': {'precision': 1.0,
  'recall': 0.9230769230769231,
  'f1-score': 0.9600000000000001,
  'support': 13},
 'IN': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 19},
 'JJ': {'precision': 1.0,
  'recall': 0.7777777777777778,
  'f1-score': 0.8750000000000001,
  'support': 9},
 'JJR': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1},
 'JJS': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1},
 'NN': {'precision': 1.0,
  'recall': 0.75,
  'f1-score': 0.8571428571428571,
  'support': 16},
 'NNP': {'precision': 0.75,
  'recall': 1.0,
  'f1-score'